In [10]:
# Re-doing Question 7 of Homework 2
import pandas as pd
import numpy as np
import statsmodels.formula.api as smf
from sklearn.neighbors import NearestNeighbors
from scipy.spatial.distance import mahalanobis
from scipy import stats

In [11]:
# Load county-level data
county_2018 = pd.read_csv("../../../data/output/county_analysis_2018.csv")

# Drop any rows with missing values in key variables
analysis_data = county_2018.dropna(subset=['bid', 'treatment', 'ffs_q2', 'ffs_q3', 'ffs_q4']).copy()

print(f"Analysis sample size: {len(analysis_data)}")
print(f"Treated (High HHI): {(analysis_data['treatment']==1).sum()}")
print(f"Control (Low HHI): {(analysis_data['treatment']==0).sum()}")

# Define variables
Y = analysis_data['bid'].values  # Outcome: plan bids
Tr = analysis_data['treatment'].values  # Treatment: 1=High HHI, 0=Low HHI
X = analysis_data[['ffs_q2', 'ffs_q3', 'ffs_q4']].values  # Covariates: FFS quartiles


Analysis sample size: 1976
Treated (High HHI): 988
Control (Low HHI): 988


In [12]:
# Estimator 1: Nearest Neighbor Matching - Inverse Variance (ATE)

def nn_matching_inv_ate(Y, Tr, X):

    treated_idx = np.where(Tr == 1)[0]
    control_idx = np.where(Tr == 0)[0]

    variances = np.var(X, axis=0)
    variances[variances == 0] = 1e-10
    X_scaled = X / np.sqrt(variances)

    # Match treated → control
    nbrs_control = NearestNeighbors(n_neighbors=1).fit(X_scaled[control_idx])
    _, indices_tc = nbrs_control.kneighbors(X_scaled[treated_idx])
    matched_control = control_idx[indices_tc.flatten()]

    # Match control → treated
    nbrs_treated = NearestNeighbors(n_neighbors=1).fit(X_scaled[treated_idx])
    _, indices_ct = nbrs_treated.kneighbors(X_scaled[control_idx])
    matched_treated = treated_idx[indices_ct.flatten()]

    Y1_hat = np.zeros(len(Y))
    Y0_hat = np.zeros(len(Y))

    Y1_hat[treated_idx] = Y[treated_idx]
    Y0_hat[treated_idx] = Y[matched_control]

    Y0_hat[control_idx] = Y[control_idx]
    Y1_hat[control_idx] = Y[matched_treated]

    ate = np.mean(Y1_hat - Y0_hat)

    return ate


ate_inv = nn_matching_inv_ate(Y, Tr, X)
print(f"1. INV (Inverse Variance Matching): ${ate_inv:.2f}")

1. INV (Inverse Variance Matching): $15.95


In [13]:
# Estimator 2: Nearest Neighbor Matching - Mahalanobis (ATE)

def nn_matching_maha_ate(Y, Tr, X):

    treated_idx = np.where(Tr == 1)[0]
    control_idx = np.where(Tr == 0)[0]

    cov_matrix = np.cov(X.T)
    inv_cov = np.linalg.pinv(cov_matrix)

    # Match treated → control
    matched_control = []
    for i in treated_idx:
        dists = [mahalanobis(X[i], X[j], inv_cov) for j in control_idx]
        matched_control.append(control_idx[np.argmin(dists)])

    # Match control → treated
    matched_treated = []
    for i in control_idx:
        dists = [mahalanobis(X[i], X[j], inv_cov) for j in treated_idx]
        matched_treated.append(treated_idx[np.argmin(dists)])

    Y1_hat = np.zeros(len(Y))
    Y0_hat = np.zeros(len(Y))

    Y1_hat[treated_idx] = Y[treated_idx]
    Y0_hat[treated_idx] = Y[matched_control]

    Y0_hat[control_idx] = Y[control_idx]
    Y1_hat[control_idx] = Y[matched_treated]

    ate = np.mean(Y1_hat - Y0_hat)

    return ate


ate_maha = nn_matching_maha_ate(Y, Tr, X)
print(f"2. MAH (Mahalanobis Matching): ${ate_maha:.2f}")

2. MAH (Mahalanobis Matching): $24.36


In [14]:
# Estimator 3: Inverse Propensity Weighting (IPW)
def ipw_estimator(Y, Tr, X):
    """
    IPW estimator following the R code
    """
    # Fit logit model for propensity score
    temp_df = pd.DataFrame({
        'treatment': Tr,
        'ffs_q2': X[:, 0],
        'ffs_q3': X[:, 1],
        'ffs_q4': X[:, 2]
    })
    
    logit_model = smf.logit('treatment ~ ffs_q2 + ffs_q3 + ffs_q4', data=temp_df).fit(disp=False)
    ps = logit_model.predict(temp_df)
    
    # Clip propensity scores to avoid extreme weights
    ps = ps.clip(0.01, 0.99)
    
    # Calculate IPW weights
    ipw = np.where(Tr == 1, 1.0 / ps, 1.0 / (1.0 - ps))
    
    # Weighted means
    mean_treated = np.average(Y[Tr == 1], weights=ipw[Tr == 1])
    mean_control = np.average(Y[Tr == 0], weights=ipw[Tr == 0])
    
    ate = mean_treated - mean_control
    
    return ate

ate_ipw = ipw_estimator(Y, Tr, X)
print(f"3. IPW (Inverse Propensity Weighting): ${ate_ipw:.2f}")

3. IPW (Inverse Propensity Weighting): $19.61


In [15]:
# Estimator 4: OLS Regression
def ols_estimator(Y, Tr, X):
    """
    OLS regression following regression adjustment approach
    Run separate regressions by treatment status, then predict for everyone
    """
    # Create dataframe for regression
    reg_df = pd.DataFrame({
        'y': Y,
        'treatment': Tr,
        'ffs_q2': X[:, 0],
        'ffs_q3': X[:, 1],
        'ffs_q4': X[:, 2]
    })
    
    # Create interactions
    reg_df['treat_q2'] = reg_df['treatment'] * reg_df['ffs_q2']
    reg_df['treat_q3'] = reg_df['treatment'] * reg_df['ffs_q3']
    reg_df['treat_q4'] = reg_df['treatment'] * reg_df['ffs_q4']
    
    # Run regression with interactions (allows separate slopes by treatment)
    ols_model = smf.ols(
        'y ~ treatment + ffs_q2 + ffs_q3 + ffs_q4 + treat_q2 + treat_q3 + treat_q4',
        data=reg_df
    ).fit()
    
    # Predict outcome for everyone AS IF treated
    reg_df_1 = reg_df.copy()
    reg_df_1['treatment'] = 1
    reg_df_1['treat_q2'] = reg_df_1['ffs_q2']
    reg_df_1['treat_q3'] = reg_df_1['ffs_q3']
    reg_df_1['treat_q4'] = reg_df_1['ffs_q4']
    pred1 = ols_model.predict(reg_df_1)
    
    # Predict outcome for everyone AS IF control
    reg_df_0 = reg_df.copy()
    reg_df_0['treatment'] = 0
    reg_df_0['treat_q2'] = 0
    reg_df_0['treat_q3'] = 0
    reg_df_0['treat_q4'] = 0
    pred0 = ols_model.predict(reg_df_0)
    
    # ATE is average difference across all units
    ate = np.mean(pred1 - pred0)
    
    return ate

ate_ols = ols_estimator(Y, Tr, X)
print(f"4. OLS (Regression with Interactions): ${ate_ols:.2f}")

4. OLS (Regression with Interactions): $19.61


In [16]:
# Table 3: Comparing the Estimators
table3 = pd.DataFrame({
    'Estimator': ['INV', 'MAH', 'IPW', 'OLS'],
    'ATE': [ate_inv, ate_maha, ate_ipw, ate_ols]
})

print("\n" + "="*70)
print("TABLE 3: Average Treatment Effects")
print("High HHI vs. Low HHI (2018 Data)")
print("="*70)
print(table3.to_string(index=False))

# Check if estimates are identical
max_diff = table3['ATE'].max() - table3['ATE'].min()
std_ate = table3['ATE'].std()

print(f"\nRange of estimates: ${max_diff:.4f}")
print(f"Standard deviation: ${std_ate:.4f}")

if max_diff < 0.50:  # Within 50 cents
    print("\n✓ SUCCESS: All estimates are essentially identical (as expected)")
    print("  This confirms all methods are using the same data and treatment definition.")
else:
    print("\n⚠ WARNING: Estimates differ substantially")
    print("  This suggests missing data or different sample sizes across methods")
    print("  Check for NaN values or inconsistent treatment assignment")


TABLE 3: Average Treatment Effects
High HHI vs. Low HHI (2018 Data)
Estimator       ATE
      INV 15.950978
      MAH 24.358274
      IPW 19.607228
      OLS 19.607228

Range of estimates: $8.4073
Standard deviation: $3.4468

⚠ WARNING: Estimates differ substantially
  This suggests missing data or different sample sizes across methods
  Check for NaN values or inconsistent treatment assignment


In [17]:
print("\n" + "="*70)
print("DIAGNOSTICS")
print("="*70)
print(f"Sample size: {len(analysis_data)}")
print(f"Treated (High HHI): {(Tr==1).sum()}")
print(f"Control (Low HHI): {(Tr==0).sum()}")
print(f"\nMissing values check:")
print(f"  Missing in bid: {analysis_data['bid'].isna().sum()}")
print(f"  Missing in treatment: {analysis_data['treatment'].isna().sum()}")
print(f"  Missing in ffs_q2: {analysis_data['ffs_q2'].isna().sum()}")
print(f"  Missing in ffs_q3: {analysis_data['ffs_q3'].isna().sum()}")
print(f"  Missing in ffs_q4: {analysis_data['ffs_q4'].isna().sum()}")


DIAGNOSTICS
Sample size: 1976
Treated (High HHI): 988
Control (Low HHI): 988

Missing values check:
  Missing in bid: 0
  Missing in treatment: 0
  Missing in ffs_q2: 0
  Missing in ffs_q3: 0
  Missing in ffs_q4: 0


In [18]:
table3.to_csv("../../../data/output/table3_question7.csv", index=False)
print("\n✓ Table saved to: ../../data/output/table3_question7.csv")

# Save the full analysis dataset 
analysis_data.to_csv("../../../data/output/question7_analysis_data.csv", index=False)
print("✓ Analysis data saved to: ../../../data/output/question7_analysis_data.csv")

print("\n" + "="*70)
print("COMPLETE!")
print("="*70)


✓ Table saved to: ../../data/output/table3_question7.csv
✓ Analysis data saved to: ../../../data/output/question7_analysis_data.csv

COMPLETE!


In [ ]:
# Redoing Question 9

print("="*70)
print("QUESTION 9: OLS Estimator with Continuous Covariates")
print("="*70)

# Using OLS 
# Prepare data with continuous covariates
continuous_data = analysis_data.dropna(subset=['bid', 'treatment', 'avg_ffscost', 'total_benef']).copy()

print(f"\nSample size with continuous covariates: {len(continuous_data)}")
print(f"Treated: {(continuous_data['treatment']==1).sum()}")
print(f"Control: {(continuous_data['treatment']==0).sum()}")

# Define variables
Y_cont = continuous_data['bid'].values
Tr_cont = continuous_data['treatment'].values

# Create dataframe for regression
reg_df_cont = pd.DataFrame({
    'y': Y_cont,
    'treatment': Tr_cont,
    'avg_ffscost': continuous_data['avg_ffscost'].values,
    'total_benef': continuous_data['total_benef'].values
})

# Create interactions between treatment and continuous covariates
reg_df_cont['treat_ffs'] = reg_df_cont['treatment'] * reg_df_cont['avg_ffscost']
reg_df_cont['treat_benef'] = reg_df_cont['treatment'] * reg_df_cont['total_benef']

# Run OLS with continuous covariates and interactions
ols_continuous = smf.ols(
    'y ~ treatment + avg_ffscost + total_benef + treat_ffs + treat_benef',
    data=reg_df_cont
).fit()

print("\nOLS Regression Results (Continuous Covariates):")
print(ols_continuous.summary().tables[1])

# Calculate ATE by predicting for everyone as treated vs control
# Predict as treated
reg_df_1 = reg_df_cont.copy()
reg_df_1['treatment'] = 1
reg_df_1['treat_ffs'] = reg_df_1['avg_ffscost']
reg_df_1['treat_benef'] = reg_df_1['total_benef']
pred1_cont = ols_continuous.predict(reg_df_1)

# Predict as control
reg_df_0 = reg_df_cont.copy()
reg_df_0['treatment'] = 0
reg_df_0['treat_ffs'] = 0
reg_df_0['treat_benef'] = 0
pred0_cont = ols_continuous.predict(reg_df_0)

# ATE is average difference
ate_ols_continuous = np.mean(pred1_cont - pred0_cont)

print(f"\n{'='*70}")
print("COMPARISON: Quartile-based vs Continuous Specification")
print(f"{'='*70}")
print(f"\nOLS with FFS Quartiles:              ${ate_ols:.2f}")
print(f"OLS with Continuous FFS + Beneficiaries: ${ate_ols_continuous:.2f}")
print(f"\nDifference:                           ${ate_ols_continuous - ate_ols:.2f}")

# Calculate percentage difference
pct_diff = ((ate_ols_continuous - ate_ols) / ate_ols) * 100
print(f"Percentage difference:                 {pct_diff:.1f}%")

# Create comparison table
comparison_table = pd.DataFrame({
    'Specification': [
        'Quartile-based (Q2, Q3, Q4)',
        'Continuous (FFS costs + Total beneficiaries)'
    ],
    'ATE': [ate_ols, ate_ols_continuous],
    'Covariates': [
        'FFS quartile dummies',
        'Continuous FFS costs, Total Medicare beneficiaries'
    ]
})

print(f"\n{'='*70}")
print("Table: OLS Estimates with Different Covariate Specifications")
print(f"{'='*70}")
print(comparison_table.to_string(index=False))

# Save results
comparison_table.to_csv(
    os.path.join(base_path, "data", "output", "question9_comparison.csv"),
    index=False
)
print(f"\n✓ Comparison table saved to: data/output/question9_comparison.csv")

# Diagnostic: Check if adding total beneficiaries improves fit
print(f"\n{'='*70}")
print("Model Diagnostics")
print(f"{'='*70}")
print(f"R-squared (continuous model): {ols_continuous.rsquared:.4f}")

# For comparison, run model without beneficiaries
ols_ffs_only = smf.ols(
    'y ~ treatment + avg_ffscost + treat_ffs',
    data=reg_df_cont
).fit()
print(f"R-squared (FFS only):        {ols_ffs_only.rsquared:.4f}")

# Check coefficient on treatment
print(f"\nTreatment coefficient (continuous model): ${ols_continuous.params['treatment']:.2f}")
print(f"Standard error:                           ${ols_continuous.bse['treatment']:.2f}")
print(f"P-value:                                  {ols_continuous.pvalues['treatment']:.4f}")